In [1]:
import pandas as pd
import numpy as np
from catnip.fla_redshift import FLA_Redshift
from sqlalchemy import null
from datetime import datetime

from prefect.blocks.system import Secret
from typing import Dict
from concurrent.futures import ThreadPoolExecutor

In [2]:
def get_redshift_credentials() -> Dict:

    cred_dict = {
        "dbname": Secret.load("stellar-redshift-db-name").get(),
        "host": Secret.load("stellar-redshift-host").get(),
        "port": 5439,
        "user": Secret.load("stellar-redshift-user-name").get(),
        "password": Secret.load("stellar-redshift-password").get(),

        "aws_access_key_id": Secret.load("fla-s3-aws-access-key-id-east-1").get(),
        "aws_secret_access_key": Secret.load("fla-s3-aws-secret-access-key-east-1").get(),
        "bucket": Secret.load("fla-s3-bucket-name-east-1").get(),
        "subdirectory": "us-east-1",

        "verbose": False,
    }

    return cred_dict

with ThreadPoolExecutor(1) as pool:
    rs_creds = pool.submit(lambda: get_redshift_credentials()).result()

In [3]:
df = pd.read_csv("C:\\Users\\riffere\\Desktop\\ALTP_Combined.csv")
FLA_Redshift(**rs_creds).write_to_warehouse(df = df, table_name= "ALTP_Combined_data")

In [4]:
q = """
WITH acct_ids AS
    (SELECT
         crm_id as acct_id, lower(email) as email, date(creation_datetime) as add_date
    FROM
        custom.seatgeek_v_clients
    UNION
    SELECT
        acct_id, lower(email_addr) as email, date(add_date) as add_date
    FROM
        custom.arch_final_v_customer),
emails_and_accounts AS
    (SELECT
         altp_combined_data.email, acct_id, CAST(MIN(season) AS VARCHAR) AS "season",
         CAST(MIN(transaction_date) AS date) AS "transaction_date", add_date
    FROM
        custom.altp_combined_data
    LEFT JOIN
        acct_ids on altp_combined_data.email = acct_ids.email
    WHERE
        altp_combined_data.email is not null and altp_combined_data.email <> ''
    GROUP BY
        altp_combined_data.email, acct_id, add_date),
historical_data AS
    (SELECT
        emails_and_accounts.acct_id, email, emails_and_accounts.season, transaction_date, emails_and_accounts.add_date,
        CASE
            WHEN cth_historical_all_1718_2223.season like '2017-18' and "ticket_type" IN ('Annual Suites', 'Full', 'Half', 'Premier') THEN 1
            ELSE 0
        END AS "is_stm_2017_18",
        CASE
            WHEN cth_historical_all_1718_2223.season like '2018-19' and "ticket_type" IN ('Annual Suites', 'Full', 'Half', 'Premier') THEN 1
            ELSE 0
        END AS "is_stm_2018_19",
        CASE
            WHEN cth_historical_all_1718_2223.season like '2019-20' and "ticket_type" IN ('Annual Suites', 'Full', 'Half', 'Premier') THEN 1
            ELSE 0
        END AS "is_stm_2019_20",
        CASE
            WHEN cth_historical_all_1718_2223.season like '2020-21' and "ticket_type" IN ('Annual Suites', 'Full', 'Half', 'Premier') THEN 1
            ELSE 0
        END AS "is_stm_2020_21",
        CASE
            WHEN cth_historical_all_1718_2223.season like '2021-22' and "ticket_type" IN ('Annual Suites', 'Full', 'Half', 'Premier') THEN 1
            ELSE 0
        END AS "is_stm_2021_22",
        CASE
            WHEN cth_historical_all_1718_2223.season like '2022-23' and "ticket_type" IN ('Annual Suites', 'Full', 'Half', 'Premier') THEN 1
            ELSE 0
        END AS "is_stm_2022_23",
        CASE
            WHEN cth_historical_all_1718_2223.season like '2017-18' then gross_revenue
            ELSE 0
        END AS "is_gross_rev_2017_18",
        CASE
            WHEN cth_historical_all_1718_2223.season like '2018-19' then gross_revenue
            ELSE 0
        END AS "is_gross_rev_2018_19",
        CASE
            WHEN cth_historical_all_1718_2223.season like '2019-20' then gross_revenue
            ELSE 0
        END AS "is_gross_rev_2019_20",
        CASE
            WHEN cth_historical_all_1718_2223.season like '2020-21' then gross_revenue
            ELSE 0
        END AS "is_gross_rev_2020_21",
        CASE
            WHEN cth_historical_all_1718_2223.season like '2021-22' then gross_revenue
            ELSE 0
        END AS "is_gross_rev_2021_22",
        CASE
            WHEN cth_historical_all_1718_2223.season like '2022-23' then gross_revenue
            ELSE 0
        END AS "is_gross_rev_2022_23"
    FROM
        emails_and_accounts
    LEFT JOIN
        custom.cth_historical_all_1718_2223 on emails_and_accounts.acct_id = cth_historical_all_1718_2223.acct_id),
current_data AS
    (SELECT
         purch_client_crm_id as acct_id, email, emails_and_accounts.season, emails_and_accounts.transaction_date,
         emails_and_accounts.add_date, gross_revenue as is_gross_rev_2023_24,
        CASE
            WHEN "ticket_type" IN ('Annual Suites', 'Full', 'Half', 'Premier') THEN 1
            ELSE 0
        END AS "is_stm_2023_24"
    FROM
        emails_and_accounts
    LEFT JOIN
        custom.cth_v_ticket_2324 on cth_v_ticket_2324.purch_client_crm_id = emails_and_accounts.acct_id),
historical_agg AS
    (SELECT
    email, season, transaction_date, add_date, max(is_stm_2017_18) as is_stm_2017_18,
    max(is_stm_2018_19) as is_stm_2018_19,
    max(is_stm_2019_20) as is_stm_2019_20, max(is_stm_2020_21) as is_stm_2020_21,
    max(is_stm_2021_22) as is_stm_2021_22, max(is_stm_2022_23) as is_stm_2022_23,
    sum(is_gross_rev_2017_18) as is_gross_rev_2017_18, sum(is_gross_rev_2018_19) as is_gross_rev_2018_19,
    sum(is_gross_rev_2019_20) as is_gross_rev_2019_20, sum(is_gross_rev_2020_21) as is_gross_rev_2020_21,
    sum(is_gross_rev_2021_22) as is_gross_rev_2021_22, sum(is_gross_rev_2022_23) as is_gross_rev_2022_23
FROM
    historical_data
GROUP BY
    email, season, transaction_date, add_date),
current_agg AS
    (SELECT
    email, season, transaction_date, add_date,
    coalesce(max(is_stm_2023_24),0) as is_stm_2023_24,
    coalesce(sum(is_gross_rev_2023_24),0) as is_gross_rev_2023_24
    FROM
        current_data
    GROUP BY
        email, season, transaction_date, add_date)
SELECT
    historical_agg.email, historical_agg.season, historical_agg.transaction_date,
    historical_agg.add_date, is_stm_2017_18, is_stm_2018_19,
    is_stm_2019_20, is_stm_2020_21, is_stm_2021_22, is_stm_2022_23, is_stm_2023_24,
    is_gross_rev_2017_18, is_gross_rev_2018_19, is_gross_rev_2019_20,
    is_gross_rev_2020_21, is_gross_rev_2021_22, is_gross_rev_2022_23, is_gross_rev_2023_24
FROM
    historical_agg
LEFT JOIN
    current_agg on historical_agg.email = current_agg.email
"""

In [5]:
df = FLA_Redshift(**rs_creds).query_warehouse(sql_string = q)

In [6]:
df['add_date'].fillna(datetime.strptime('2030-01-01', "%Y-%m-%d"), inplace = True)
df['add_date'] = pd.to_datetime(df['add_date']).dt.date

In [7]:
df['pre_altp_email'] = df.apply(lambda row: 0 if row['transaction_date'] == '' else (1 if row['add_date'] < row['transaction_date'] else 0), axis = 1)
df['post_altp_email'] = df.apply(lambda row: 0 if row['transaction_date'] == '' else (1 if row['add_date'] >= row['transaction_date'] else 0), axis = 1)

In [8]:
season_dict = {
    "2017-18" : 0,
    "2018-19" : 1,
    "2019-20" : 2,
    "2020-21" : 3,
    "2021-22" : 4,
    "2022-23" : 5,
    "2023-24" : 6
}

# sum season tickets before/after
for key, value in season_dict.items():
    col_list = list(df.columns[4:(4+value)])
    # print(col_list)
    df.loc[(df['season'] == key), 'pre_ltp_ticktets'] =  df[col_list].sum(axis = 1)

for key, value in season_dict.items():
    col_list = list(df.columns[(4 + value):10])
    # print(col_list)
    df.loc[(df['season'] == key), 'post_ltp_tickets'] =  df[col_list].sum(axis = 1)

# sum revenue before/after
for key, value in season_dict.items():
    col_list = list(df.columns[11:(11+value)])
    # print(col_list)
    df.loc[(df['season'] == key), 'pre_ltp_revenue'] =  df[col_list].sum(axis = 1)


for key, value in season_dict.items():
    col_list = list(df.columns[(11 + value):17])
    # print(col_list)
    df.loc[(df['season'] == key), 'post_ltp_revenue'] =  df[col_list].sum(axis = 1)

In [9]:
FLA_Redshift(**rs_creds).write_to_warehouse(df = df, table_name= "altp_type_data")